# Read data

In [ ]:
import xarray as xr

In [ ]:
isl        = 1
YYYY_list  = [2012];
nYYYY      = len(YYYY_list)

In [ ]:
datadir         = '/home/disk/eos12/wycheng/data'

island_file     = f'{datadir}/US/island/island_1deg_US.nc'
island_data     = xr.open_dataset(island_file)

for iyear in range(nYYYY):
    YYYY        = str(YYYY_list[iyear])
    F_file      = f'{datadir}/US/WWLLN/WWLLN_{YYYY}_F_cg_1deg3hr_US.nc'
    cape_file   = f'{datadir}/US/ERA5/ERA5_{YYYY}_cape_cg_1deg3hr_US.nc'
    pcp_file    = f'{datadir}/US/TRMM/TRMM_{YYYY}_pcp_cg_1deg3hr_US.nc'
    
    F_data      = xr.open_dataset(F_file)
    cape_data   = xr.open_dataset(cape_file)
    pcp_data    = xr.open_dataset(pcp_file)

In [ ]:
pcp_data

# Plots

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as mticker

In [ ]:
plt.rcParams.update({'font.size': 48})
figsize = (40,20)


In [ ]:
def plot_map(figsize,data,cmap,vmin=None,vmax=None,title=None,unit=None):
    
    fig   = plt.figure(figsize=figsize)

    pcm = data.plot.pcolormesh("lon","lat",
        subplot_kws=dict(projection=ccrs.PlateCarree(), facecolor="white"),
        transform=ccrs.PlateCarree(),
        cmap=cmap,
        vmin=vmin,vmax=vmax,
        add_colorbar = False,
    )
    pcm.axes.set_extent([-135, -60, 20, 60], ccrs.PlateCarree())
    gl = pcm.axes.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=2, color='gray', alpha=0.5, linestyle='--')
    gl.xlocator = mticker.FixedLocator([-130, -120, -110, -100, -90, -80, -70, -60])
    gl.ylocator = mticker.FixedLocator([20, 30, 40, 50, 60])
    gl.xlabels_top   = False
    gl.ylabels_right = False
    pcm.axes.coastlines()
    plt.title(title)

    divider = make_axes_locatable(pcm.axes)
    cax     = divider.append_axes("right", size="2%", pad=1, axes_class=plt.Axes)
    cbar    = plt.colorbar(pcm, cax=cax,extend='max')
    plt.text(vmin,vmax*1.08,unit)

In [ ]:
data  = (1/((111.19492664455873)**2)) * (365.25*8) * F_data['F'].mean(dim='Time').where(island_data.island==1)
cmap  = plt.get_cmap('jet')
vmin  = 0
vmax  = 10
title = 'Lightning flash rate'
unit  = '[km$^{-2}$ yr$^{-1}$]'

plot_map(figsize,data=data,cmap=cmap,vmin=vmin,vmax=vmax,title=title,unit=unit)

In [ ]:
data  = pcp_data['pcp'].mean(dim='time').where(island_data.island==1)
cmap  = plt.get_cmap('jet')
vmin  = 0
vmax  = 0.2
title = 'Precipitation'
unit  = '[mm hr$^{-1}$]'

plot_map(figsize,data=data,cmap=cmap,vmin=vmin,vmax=vmax,title=title,unit=unit)

In [ ]:
data  = cape_data['cape'].mean(dim='Time').where(island_data.island==1)
cmap  = plt.get_cmap('jet')
vmin  = 0
vmax  = 800
title = 'CAPE'
unit  = '[J kg$^{-1}$]'

plot_map(figsize,data=data,cmap=cmap,vmin=vmin,vmax=vmax,title=title,unit=unit)

In [ ]:
data  = F_data['F'] \
        .where(F_data.F>0) \
        .count(dim='Time') \
        .where(island_data.island==1)
cmap  = plt.get_cmap('jet')
vmin  = 0
vmax  = 500
title = 'Lightning events'
unit  = '[#]'

plot_map(figsize,data=data,cmap=cmap,vmin=vmin,vmax=vmax,title=title,unit=unit)

# Dry Thunderstorm

In [ ]:
data  = F_data['F'] \
        .where((F_data.F>0) & (pcp_data.pcp<(2.5/3))) \
        .count(dim='Time') \
        .where(island_data.island==1)
cmap  = plt.get_cmap('jet')
vmin  = 0
vmax  = 300
title = 'Dry thunderstorm events'
unit  = '[#]'

plot_map(figsize,data=data,cmap=cmap,vmin=vmin,vmax=vmax,title=title,unit=unit)

In [ ]:
data  = (F_data['F'] \
        .where(F_data.F>0 & pcp_data.pcp<(2.5/3)) \
        .count(dim='Time') \
        .where(island_data.island==1))@
        (1/F_data['F'] \
        .where(F_data.F>0) \
        .count(dim='Time') \
        .where(island_data.island==1))@

cmap  = plt.get_cmap('jet')
vmin  = 0
vmax  = 300
title = 'Dry thunderstorm events'
unit  = '[#]'

plot_map(figsize,data=data,cmap=cmap,vmin=vmin,vmax=vmax,title=title,unit=unit)